### Getting Setup (On Google Colab)

* Begin by installing some pip packages and the java development kit.

In [1]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet
!apt install openjdk-8-jdk-headless &> /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [14]:
#RDD -- Resilient Distributed Datasets

In [2]:
import os
os.environ["JAVA_HOME"] = "/lib/jvm/java-8-openjdk-amd64"


In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()


In [6]:
#1: map() Transformation
ex_rdd = sc.parallelize([5, 10, 15, 20])
print("Result of map():", ex_rdd.map(lambda x: x * 2).collect())

Result of map(): [10, 20, 30, 40]


In [7]:
#2: filter() Transformation
# ----------------------------
f_rdd = sc.parallelize([12, 19, 25, 28, 33, 40])
print("Result of filter():", f_rdd.filter(lambda x: x > 20).collect())


Result of filter(): [25, 28, 33, 40]


In [8]:
#3: union() of Two Filtered RDDs

input_rdd = sc.parallelize([15, 22, 30, 37, 44, 50, 63])

u_rdd1 = input_rdd.filter(lambda x: x % 5 == 0)   # divisible by 5
u_rdd2 = input_rdd.filter(lambda x: x % 7 == 0)   # divisible by 7

print("Result of union():", u_rdd1.union(u_rdd2).collect())

Result of union(): [15, 30, 50, 63]


In [9]:
#4: flatMap() Transformation

fmap_rdd = sc.parallelize(["Data Science is fun", "PySpark makes big data easy"])
print("Result of flatMap():", fmap_rdd.flatMap(lambda x: x.split(" ")).collect())


Result of flatMap(): ['Data', 'Science', 'is', 'fun', 'PySpark', 'makes', 'big', 'data', 'easy']


In [10]:
#1: collect() Action

clct_rdd = sc.parallelize([9, 18, 27, 36, 45])
print("Result of collect():", clct_rdd.collect())

Result of collect(): [9, 18, 27, 36, 45]


In [11]:
#2: count() Action

cnt_rdd = sc.parallelize([5, 7, 9, 11, 13, 15, 17, 19])
print("Result of count():", cnt_rdd.count())

Result of count(): 8


In [12]:
#3: take() Action

tk_rdd = sc.parallelize([101, 202, 303, 404, 505])
print("Result of take(3):", tk_rdd.take(3))

Result of take(3): [101, 202, 303]


In [37]:
#4: reduce() Action

red_rdd = sc.parallelize([4, 8, 12, 16, 20])
print("Result of reduce():", red_rdd.reduce(lambda x, y: x + y))

Result of reduce(): 60


In [38]:
#5: saveAsTextFile() Action

sv_rdd = sc.parallelize([10, 20, 30, 40])
sv_rdd.saveAsTextFile("output_textfile")
print("RDD saved as text file in 'output_textfile' folder")

RDD saved as text file in 'output_textfile' folder


In [39]:
# Stop Spark context (optional)
sc.stop()


* Then connect to a SparkSession, setting the spark ui port to `4050`.

In [40]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').setAppName("films").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf=conf)

* Then we need to install ngrok which will allow us to place our local spark ui on the web.

In [41]:
from pyspark import SparkContext, SparkConf

# Stop any existing SparkContext
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

# Now safely create a new one
conf = SparkConf().set('spark.ui.port', '4050').setAppName("films").setMaster("local[*]")
sc = SparkContext.getOrCreate(conf=conf)


Now  let's again create an RDD from our movie records.

In [42]:
movies = ['dark knight', 'dunkirk', 'pulp fiction', 'avatar']

In [43]:
movies_rdd = sc.parallelize(movies)
movies_rdd

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

And then let's capitalize the movies, and select the movies that begin with `d`.

In [44]:
movies_rdd.collect() #action

['dark knight', 'dunkirk', 'pulp fiction', 'avatar']

In [45]:
movies_rdd.take(3) #actions


['dark knight', 'dunkirk', 'pulp fiction']

In [46]:
movies[0].title()

'Dark Knight'

In [47]:
movies

['dark knight', 'dunkirk', 'pulp fiction', 'avatar']

In [48]:
transform=lambda i:i.title()
movies_title=[transform(i) for i in movies] #not pyspark operation

In [49]:
movies_title

['Dark Knight', 'Dunkirk', 'Pulp Fiction', 'Avatar']

In [50]:
movies_title_rdd=movies_rdd.map(transform) ## transformation

In [51]:
movies_title_rdd.collect()

['Dark Knight', 'Dunkirk', 'Pulp Fiction', 'Avatar']

In [52]:
movies_rdd.collect()

['dark knight', 'dunkirk', 'pulp fiction', 'avatar']

In [53]:
movies_rdd.filter(lambda movies : movies[0]=='d').collect()

['dark knight', 'dunkirk']

In [54]:
movies_rdd.map(lambda movie: movie.title()).take(2)
#transformations -- lazy transformations
## once you apply a transformation only the function is created but it is not applied
## you need an action to apply the transformation across your rdd

['Dark Knight', 'Dunkirk']

In [60]:

rdd2.collect()

['Dark Knight', 'Dunkirk', 'Pulp Fiction', 'Avatar']

In [61]:
rdd1=movies_rdd.map(lambda movies :movies.title()).collect()

In [62]:
type(rdd1)

list

In [65]:
rdd2=movies_rdd.map(lambda movies : movies.title())

In [66]:
type(rdd2)

pyspark.rdd.PipelinedRDD

In [67]:
movies_rdd.map(lambda movies :movies.title()).collect()

['Dark Knight', 'Dunkirk', 'Pulp Fiction', 'Avatar']

Now as we know, Spark will partition the dataset across the cores of the executors, and then map through the records in parallel, returning all of the results.

> <img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/parallel.png?raw=1" width="60%">

Now let's change the function so that this time, instead of returning all of the results, we just return the first result.

In [68]:
movies_rdd.map(lambda movie: movie.title()).take(1)


['Dark Knight']

Now if we think about, this previous step, here we would not have to map through all of the steps just to return a single result.  And it turns out if we look at Spark, we can see that even though the dataset was distributed -- it only needed to perform work on a single partition to return one result.

> <img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/individual_task.png?raw=1" width="80%">

This ability, to see the end result that needs to be returned, and to work efficiently to only take the needed steps to return those results, is a valuable feature when working with large datasets.  And we can better see how Spark accomplishes it in the next section.

### A little experiment

If we run the code below, notice that nothing is returned.

In [69]:
movies_rdd.map(lambda movie: movie.title())

PythonRDD[11] at RDD at PythonRDD.scala:53

And even if we chain the map and the filter methods, still nothing is returned.

In [70]:
movies_rdd.map(lambda movie: movie.title()).filter(lambda movie: movie[0] == 'D').collect()

['Dark Knight', 'Dunkirk']

It's only when we add a collect function on the end, will some data be returned.

In [71]:
movies_rdd.filter(lambda movie: movie[0] == 'd').map(lambda movie: movie.title()).collect()

['Dark Knight', 'Dunkirk']

So above, nothing was returned when we ran the `map` and `collect` functions, because when we only executed those functions, Spark did not actually act on the data.  Then in the third line we finally did act on the data.  We told Spark that we want to both transform, and filter the data, and then return all of the results.  

So it's only when we called the `collect` function that Spark's driver determined the tasks to then send off to the executors and return the results.

### Transformations and Actions

So above we can see that the functions `map` and `filter` do not actually perform any work on our data.  Instead steps are only kicked off when we call the `collect` method.  

In Spark, the methods that kick off tasks and return results are called **actions** (eg. collect).  And methods like `map` and `transform` that do not are called **transformations**.  

1. Transformations

So we already saw that transformations include `map` and `filter`, and our transformations do not actually return results to our users.  Here's a couple other transformations.

* sample

The `sample` method allows us to take a random sample from our dataset.  

In [72]:
movies_rdd.sample(fraction = 0.5, withReplacement = False).collect()

['dunkirk']

> Notice that it does not return any data.

* distinct

In [73]:
movies_rdd.distinct().collect()

['dark knight', 'avatar', 'dunkirk', 'pulp fiction']

> Distinct finds the unique results.  Notice that it also does not return data.

Finally, we have already seen `map`, which provides a one to one transformation of our records, and `select` which filters our data.  In each case, our transformations do not return data to us.

2. Actions

Actions are a bit more about the end result.  So far we've learned about `collect`, which returns *all* of the results of a series of transformations.  

* Take

We've also seen `take`, which limits our results to a subset.

In [74]:
movies_rdd.distinct().take(2)

['dark knight', 'avatar']

> So `take` is similar to the `LIMIT` function in SQL. Notice that here our records are returned.

* Count

In [75]:
movies_rdd.distinct().count()

4

Count simply counts the results.